#### AMYPred-FRL is a novel approach for accurate prediction of amyloid proteins by using feature representation learning
Charoenkwan, P., Ahmed, S., Nantasenamat, C. et al. AMYPred-FRL is a novel approach for accurate prediction of amyloid proteins by using feature representation learning. Sci Rep 12, 7697 (2022). https://doi.org/10.1038/s41598-022-11897-z


In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
""" Created on Mon Dec 16 14: 23:14 2019
@author
: logistics
"""
import os
for dirname, _, filenames in os.walk('/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
import re, os, sys
from collections import Counter
import math
import numpy as np
import re
import pandas as pd
from sklearn.metrics import roc_auc_score

You can modify the read_protein_sequences function to also accept a string as input by adding an optional parameter called input_string with a default value of None. You can then check whether the input_string is provided or not, and if it is provided, use it instead of reading the file. Here's the modified function:

In [2]:
# Define the function read_protein_sequences with a single argument 'file'
def read_protein_sequences(file=None, input_string=None):
    if file and input_string:
        raise ValueError("Both 'file' and 'input_string' cannot be used at the same time.")
    if not file and not input_string:
        raise ValueError("Either 'file' or 'input_string' must be provided.")

    if file:
        if os.path.exists(file) == False:
            print('Error: file %s does not exist.' % file)
            sys.exit(1)

        with open(file) as f:
            records = f.read()
    else:
        records = input_string
    
    if re.search('>', records) == None:
        print('Error: the input seems not in FASTA format!')
        sys.exit(1)


    # Split the records into individual FASTA sequences
    records = records.split('>')[1:]

    # Initialize an empty list to store the extracted sequences
    fasta_sequences = []

    # Iterate through the records and process each sequence
    for fasta in records:
        # Split the sequence by newline characters
        array = fasta.split('\n')
        # Extract the header and the sequence, replacing any non-standard amino acids with '-'
        header, sequence = array[0].split()[0], re.sub('[^ACDEFGHIKLMNPQRSTVWY-]', '-', ''.join(array[1:]).upper())
        # Split the header using the '|' character
        header_array = header.split('|')
        # Extract the name from the header
        name = header_array[0]
        # Append the name and sequence as a list to the fasta_sequences list
        fasta_sequences.append([name, sequence])

    # Return the list of fasta_sequences
    return fasta_sequences


# Feature Extraction Methods
I'll provide a brief explanation of each of these feature extraction methods, which are primarily used in bioinformatics and chemoinformatics to analyze protein or compound sequences:

AAC (Amino Acid Composition): AAC calculates the relative frequency of each amino acid in a protein sequence. This method provides a simple representation of a protein's composition by generating a 20-dimensional feature vector.

DPC (Dipeptide Composition): DPC calculates the frequency of all possible dipeptide combinations (400 total) in a protein sequence. This approach captures local sequence-order information and results in a 400-dimensional feature vector.

APAAC (Amphiphilic Pseudo Amino Acid Composition): APAAC combines AAC with amphiphilic properties of amino acids. The method considers the hydrophobicity and hydrophilicity patterns of protein sequences, resulting in a higher-dimensional feature vector that captures both composition and physicochemical properties.

CTDC (Composition, Transition, and Distribution of Chemical Groups): CTDC is a feature extraction method for protein sequences that incorporates the composition, transition, and distribution of different chemical groups (e.g., polar, nonpolar) within the sequence.

CTDD (Composition, Transition, and Distribution of Dipeptides): CTDD is similar to CTDC, but it focuses on dipeptides. It captures the composition, transition, and distribution of dipeptide groups within the protein sequence.

CTDT (Composition, Transition, and Distribution of Tripeptides): CTDT extends the concept of CTDC and CTDD to tripeptides. It calculates the composition, transition, and distribution of tripeptide groups within the protein sequence.

GAAC (Grouped Amino Acid Composition): GAAC groups amino acids according to their physicochemical properties (e.g., size, charge, hydrophobicity) and calculates the relative frequency of these groups in the protein sequence. This approach results in a lower-dimensional feature vector compared to AAC.

KSCTriad (K-Spaced Amino Acid Pairs Composition with Triads): KSCTriad combines the K-spaced amino acid pairs composition (which considers pairs of amino acids separated by K residues) with a triad-based method that captures local sequence information. This results in a high-dimensional feature vector.

CTriad (Conjoint Triad): CTriad represents a protein sequence by considering the frequency of overlapping triads of amino acids. It groups amino acids based on their physicochemical properties and encodes the sequence using a 343-dimensional feature vector.

DDE (Dragon Descriptor Extraction): DDE is a chemoinformatics approach that extracts molecular descriptors for chemical compounds. Dragon is a software package that calculates a wide range of molecular descriptors (e.g., topological, geometrical, electronic, and thermodynamic properties) to characterize compounds.

PAAC (Pseudo Amino Acid Composition): PAAC is a protein feature extraction method that combines the amino acid composition with additional sequence-order information. The resulting feature vector captures both the composition and the physicochemical properties of the protein sequence.

These methods help analyze protein or compound sequences, enabling researchers to study their properties, classify them, and predict their function or interactions.

Define the 20 amino acids.
2-3. Initialize empty lists encodings and header.
4-6. Loop through amino acids and append each amino acid to the header list.
7-16. Loop through protein sequences in fastas, removing gaps, counting amino acid occurrences, and computing their relative frequencies. Append the computed frequencies to the encodings list.
Return the encodings NumPy array and the header.

In [3]:
#This function computes the amino acid composition of the given protein sequences. It takes a list of protein sequences as input, calculates the relative frequency of each amino acid in the sequence, and returns a NumPy array of the amino acid compositions along with their headers.
def AAC(fastas, **kw):
    AA = 'ACDEFGHIKLMNPQRSTVWY'
    #AA = 'ARNDCQEGHILKMFPSTWYV'
    encodings = []
    header = []
    for i in AA:
        header.append(i)
    #encodings.append(header)

    for i in fastas:
        name, sequence = i[0], re.sub('-', '', i[1])
        count = Counter(sequence)
        for key in count:
            count[key] = count[key]/len(sequence)
        code = []
        for aa in AA:
            code.append(count[aa])
        encodings.append(code)
    return np.array(encodings, dtype=float), header

Define the 20 amino acids.
Read the AAindex data file.
3-9. Parse the AAindex data file and store the property values and names in the lists AAindex and AAindexName.
10-16. If user-defined properties are provided, filter the parsed AAindex data to only include the specified properties.
17-19. Loop through AAindex names and append each property name to the header list.
20-35. Loop through protein sequences in fastas and compute the average property value for each sequence. Append the computed values to the encodings list.
Return the encodings NumPy array and the header.

In [4]:
# This function computes the AAINDEX features, which are based on various physicochemical and biological properties of amino acids. It takes a list of protein sequences and an optional list of properties as input, calculates the average property value for each protein sequence, and returns a NumPy array of the AAINDEX features along with their headers.
def AAINDEX(fastas, props=None, **kw):
    AA = 'ARNDCQEGHILKMFPSTWYV'
    fileAAindex = 'input/data/AAindex.txt'
    with open(fileAAindex) as f:
        records = f.readlines()[1:]

    AAindex = []
    AAindexName = []
    for i in records:
        AAindex.append(i.rstrip().split()[1:] if i.rstrip() != '' else None)
        AAindexName.append(i.rstrip().split()[0] if i.rstrip() != '' else None)

    index = {}
    for i in range(len(AA)):
        index[AA[i]] = i

    #  use the user inputed properties
    if props:
        tmpIndexNames = []
        tmpIndex = []
        for p in props:
            if AAindexName.index(p) != -1:
                tmpIndexNames.append(p)
                tmpIndex.append(AAindex[AAindexName.index(p)])
        if len(tmpIndexNames) != 0:
            AAindexName = tmpIndexNames
            AAindex = tmpIndex

    header = []
    for idName in AAindexName:
        header.append(idName)

    encodings = []
    for i in fastas:
        name, sequence = i[0], i[1]
        code = []

        for j in AAindex:
            tmp = 0
            for aa in sequence:
                if aa == '-':
                    tmp = tmp + 0
                else:
                    tmp = tmp + float(j[index[aa]])
            code.append(tmp/len(sequence))
        encodings.append(code)
    return np.array(encodings, dtype=float), header

Define the data file for pseudo amino acid composition properties.
2-9. Read and parse the data file, storing amino acid properties and property names in the lists AAProperty and AAPropertyNames.
10-13. Normalize the amino acid properties.
14-20. Initialize empty lists encodings and header. Loop through amino acids and property names, appending them to the header list.
21-38. Loop through protein sequences in fastas, removing gaps, and compute the correlation factors between amino acid properties and sequence positions. Append the computed factors to the encodings list.
Return the encodings NumPy array and the header.

In [5]:
# This function computes the amphiphilic pseudo amino acid composition of the given protein sequences. It takes a list of protein sequences, an optional lambda value (default 10), and an optional weight factor (default 0.05) as input. The function calculates the correlation factors between amino acid properties and sequence positions, and returns a NumPy array of the APAAC features along with their headers.
def APAAC(fastas, lambdaValue=10, w=0.05, **kw):
    dataFile = 'data/PAAC.txt'

    with open(dataFile) as f:
        records = f.readlines()
    AA = ''.join(records[0].rstrip().split()[1:])
    AADict = {}
    for i in range(len(AA)):
        AADict[AA[i]] = i
    AAProperty = []
    AAPropertyNames = []
    for i in range(1, len(records) - 1):
        array = records[i].rstrip().split() if records[i].rstrip() != '' else None
        AAProperty.append([float(j) for j in array[1:]])
        AAPropertyNames.append(array[0])

    AAProperty1 = []
    for i in AAProperty:
        meanI = sum(i) / 20
        fenmu = math.sqrt(sum([(j - meanI) ** 2 for j in i]) / 20)
        AAProperty1.append([(j - meanI) / fenmu for j in i])

    encodings = []
    header = []
    for i in AA:
        header.append('Pc1.' + i)
    for j in range(1, lambdaValue + 1):
        for i in AAPropertyNames:
            header.append('Pc2.' + i + '.' + str(j))

    for i in fastas:
        name, sequence = i[0], re.sub('-', '', i[1])
        code = []
        theta = []

        for n in range(1, lambdaValue + 1):
            for j in range(len(AAProperty1)):
                theta.append(sum([AAProperty1[j][AADict[sequence[k]]] * AAProperty1[j][AADict[sequence[k + n]]] for k in
                                  range(len(sequence) - n)]) / (len(sequence) - n))
        myDict = {}
        for aa in AA:
            myDict[aa] = sequence.count(aa)

        code = code + [myDict[aa] / (1 + w * sum(theta)) for aa in AA]
        code = code + [w * value / (1 + w * sum(theta)) for value in theta]

        encodings.append(code)
    return np.array(encodings, dtype=float), header

In [6]:
# ASA required SPINEX external program
# BINARY encoding required same protein sequence length

# This function computes the BLOSUM62 encoding for a given set of protein sequences. BLOSUM62 is a substitution matrix representing the probabilities of amino acid substitutions in proteins. The input fastas is a list of tuples containing the protein name and sequence, and the function returns a numpy array of the computed encodings and a list of headers.
def BLOSUM62(fastas, **kw):
    AA = 'ARNDCQEGHILKMFPSTWYV'
    blosum62 = {
        'A': [4,  -1, -2, -2, 0,  -1, -1, 0, -2,  -1, -1, -1, -1, -2, -1, 1,  0,  -3, -2, 0],  # A
        'R': [-1, 5,  0,  -2, -3, 1,  0,  -2, 0,  -3, -2, 2,  -1, -3, -2, -1, -1, -3, -2, -3], # R
        'N': [-2, 0,  6,  1,  -3, 0,  0,  0,  1,  -3, -3, 0,  -2, -3, -2, 1,  0,  -4, -2, -3], # N
        'D': [-2, -2, 1,  6,  -3, 0,  2,  -1, -1, -3, -4, -1, -3, -3, -1, 0,  -1, -4, -3, -3], # D
        'C': [0,  -3, -3, -3, 9,  -3, -4, -3, -3, -1, -1, -3, -1, -2, -3, -1, -1, -2, -2, -1], # C
        'Q': [-1, 1,  0,  0,  -3, 5,  2,  -2, 0,  -3, -2, 1,  0,  -3, -1, 0,  -1, -2, -1, -2], # Q
        'E': [-1, 0,  0,  2,  -4, 2,  5,  -2, 0,  -3, -3, 1,  -2, -3, -1, 0,  -1, -3, -2, -2], # E
        'G': [0,  -2, 0,  -1, -3, -2, -2, 6,  -2, -4, -4, -2, -3, -3, -2, 0,  -2, -2, -3, -3], # G
        'H': [-2, 0,  1,  -1, -3, 0,  0,  -2, 8,  -3, -3, -1, -2, -1, -2, -1, -2, -2, 2,  -3], # H
        'I': [-1, -3, -3, -3, -1, -3, -3, -4, -3, 4,  2,  -3, 1,  0,  -3, -2, -1, -3, -1, 3],  # I
        'L': [-1, -2, -3, -4, -1, -2, -3, -4, -3, 2,  4,  -2, 2,  0,  -3, -2, -1, -2, -1, 1],  # L
        'K': [-1, 2,  0,  -1, -3, 1,  1,  -2, -1, -3, -2, 5,  -1, -3, -1, 0,  -1, -3, -2, -2], # K
        'M': [-1, -1, -2, -3, -1, 0,  -2, -3, -2, 1,  2,  -1, 5,  0,  -2, -1, -1, -1, -1, 1],  # M
        'F': [-2, -3, -3, -3, -2, -3, -3, -3, -1, 0,  0,  -3, 0,  6,  -4, -2, -2, 1,  3,  -1], # F
        'P': [-1, -2, -2, -1, -3, -1, -1, -2, -2, -3, -3, -1, -2, -4, 7,  -1, -1, -4, -3, -2], # P
        'S': [1,  -1, 1,  0,  -1, 0,  0,  0,  -1, -2, -2, 0,  -1, -2, -1, 4,  1,  -3, -2, -2], # S
        'T': [0,  -1, 0,  -1, -1, -1, -1, -2, -2, -1, -1, -1, -1, -2, -1, 1,  5,  -2, -2, 0],  # T
        'W': [-3, -3, -4, -4, -2, -2, -3, -2, -2, -3, -2, -3, -1, 1,  -4, -3, -2, 11, 2,  -3], # W
        'Y': [-2, -2, -2, -3, -2, -1, -2, -3, 2,  -1, -1, -2, -1, 3,  -3, -2, -2, 2,  7,  -1], # Y
        'V': [0,  -3, -3, -3, -1, -2, -2, -3, -3, 3,  1,  -2, 1,  -1, -2, -2, 0,  -3, -1, 4],  # V
        '-': [0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],  # -
    }
    encodings = []
    header = []
    for i in range(0,20):
        header.append('blosum62.F'+str(AA[i]))

    for i in fastas:
        name, sequence = i[0], i[1]
        code = np.asarray([0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])
        for aa in sequence:
            code = code + np.asarray(blosum62[aa])
        encodings.append(list(code/len(sequence)))
    return np.array(encodings, dtype=float), header

In [7]:
# This function generates group pairs for a given group of amino acids. The input groupKey is a list of keys representing groups of amino acids, and the function returns a dictionary with group pairs as keys and their initial values set to 0.
def generateGroupPairs(groupKey):
    gPair = {}
    for key1 in groupKey:
        for key2 in groupKey:
            gPair[key1 + '.' + key2] = 0
    return gPair

In [8]:
# This function computes the Composition of k-Spaced Amino Acid Group Pairs (CKSAAGP) encoding for a given set of protein sequences. The input fastas is a list of tuples containing the protein name and sequence, and the function returns a numpy array of the computed encodings and a list of headers.
def CKSAAGP(fastas, gap=3, **kw):
    if gap < 0:
        print('Error: the gap should be equal or greater than zero' + '\n\n')
        return 0

    group = {
        'alphaticr': 'GAVLMI',
        'aromatic': 'FYW',
        'postivecharger': 'KRH',
        'negativecharger': 'DE',
        'uncharger': 'STCPNQ'
    }

    AA = 'ARNDCQEGHILKMFPSTWYV'

    groupKey = group.keys()

    index = {}
    for key in groupKey:
        for aa in group[key]:
            index[aa] = key

    gPairIndex = []
    for key1 in groupKey:
        for key2 in groupKey:
            gPairIndex.append(key1 + '.' + key2)

    encodings = []
    header = []
    for g in range(gap + 1):
        for p in gPairIndex:
            header.append(p + '.gap' + str(g))

    for i in fastas:
        name, sequence = i[0], re.sub('-', '', i[1])
        code = []
        for g in range(gap + 1):
            gPair = generateGroupPairs(groupKey)
            sum = 0
            for p1 in range(len(sequence)):
                p2 = p1 + g + 1
                if p2 < len(sequence) and sequence[p1] in AA and sequence[p2] in AA:
                    gPair[index[sequence[p1]] + '.' + index[sequence[p2]]] = gPair[index[sequence[p1]] + '.' + index[
                        sequence[p2]]] + 1
                    sum = sum + 1

            if sum == 0:
                for gp in gPairIndex:
                    code.append(0)
            else:
                for gp in gPairIndex:
                    code.append(gPair[gp] / sum)
        encodings.append(code)
    return np.array(encodings, dtype=float), header

In [9]:
# This function computes the Group-based Dipeptide Composition (GDPC) encoding for a given set of protein sequences. The input fastas is a list of tuples containing the protein name and sequence, and the function returns a numpy array of the computed encodings and a list of headers.
def GDPC(fastas, **kw):
    group = {
        'alphaticr': 'GAVLMI',
        'aromatic': 'FYW',
        'postivecharger': 'KRH',
        'negativecharger': 'DE',
        'uncharger': 'STCPNQ'
    }

    groupKey = group.keys()
    baseNum = len(groupKey)
    dipeptide = [g1 + '.' + g2 for g1 in groupKey for g2 in groupKey]

    index = {}
    for key in groupKey:
        for aa in group[key]:
            index[aa] = key

    encodings = []
    header = []#+dipeptide
    #encodings.append(header)

    for i in fastas:
        name, sequence = i[0], re.sub('-', '', i[1])

        code = [name]
        myDict = {}
        for t in dipeptide:
            myDict[t] = 0

        sum = 0
        for j in range(len(sequence) - 2 + 1):
            myDict[index[sequence[j]] + '.' + index[sequence[j + 1]]] = myDict[index[sequence[j]] + '.' + index[
                sequence[j + 1]]] + 1
            sum = sum + 1

        if sum == 0:
            for t in dipeptide:
                code.append(0)
        else:
            for t in dipeptide:
                code.append(myDict[t] / sum)
        encodings.append(code)

    return np.array(encodings, dtype=float), header

In [10]:
# This function computes the Group-based Amino Acid Composition (GAAC) encoding for a given set of protein sequences. The input fastas is a list of tuples containing the protein name and sequence, and the function returns a numpy array of the computed encodings and a list of headers.
def GAAC(fastas, **kw):
    group = {
        'alphatic': 'GAVLMI',
        'aromatic': 'FYW',
        'postivecharge': 'KRH',
        'negativecharge': 'DE',
        'uncharge': 'STCPNQ'
    }

    groupKey = group.keys()

    encodings = []
    header = []
    for key in groupKey:
        header.append(key)


    for i in fastas:
        name, sequence= i[0], re.sub('-', '', i[1])
        code = [name]
        count = Counter(sequence)
        myDict = {}
        for key in groupKey:
            for aa in group[key]:
                myDict[key] = myDict.get(key, 0) + count[aa]

        for key in groupKey:
            code.append(myDict[key]/len(sequence))
        encodings.append(code)

    return np.array(encodings, dtype=float), header

# This function computes the Composition of k-Spaced Amino Acid Pairs (CKSAAP) encoding for a given set of protein sequences. The input fastas is a list of tuples containing the protein name and sequence, and the function returns a numpy array of the computed encodings and a list of headers.
def CKSAAP(fastas, gap=3, **kw):
    if gap < 0:
        print('Error: the gap should be equal or greater than zero' + '\n\n')
        return 0

    AA = 'ACDEFGHIKLMNPQRSTVWY'
    encodings = []
    aaPairs = []
    for aa1 in AA:
        for aa2 in AA:
            aaPairs.append(aa1 + aa2)

    header = []
    for g in range(gap + 1):
        for aa in aaPairs:
            header.append(aa + '.gap' + str(g))

    for i in fastas:
        name, sequence = i[0], i[1]
        code = []
        for g in range(gap + 1):
            myDict = {}
            for pair in aaPairs:
                myDict[pair] = 0
            sum = 0
            for index1 in range(len(sequence)):
                index2 = index1 + g + 1
                if index1 < len(sequence) and index2 < len(sequence) and sequence[index1] in AA and sequence[
                    index2] in AA:
                    myDict[sequence[index1] + sequence[index2]] = myDict[sequence[index1] + sequence[index2]] + 1
                    sum = sum + 1
            for pair in aaPairs:
                code.append(myDict[pair] / sum)
        encodings.append(code)
    return np.array(encodings, dtype=float), header

In [11]:
# This function takes two sequences as input, and returns the total number of occurrences of elements in seq1 in seq2. It initializes a sum variable to zero and iterates through the elements in seq1, incrementing the sum by the count of that element in seq2.
def Count(seq1, seq2):
    sum = 0
    for aa in seq1:
        sum = sum + seq2.count(aa)
    return sum

In [12]:
# This function calculates the amino acid composition properties of the sequences in fastas. It first defines groups of amino acids by their properties, such as charged, aliphatic, aromatic, etc. Then, it calculates the proportion of each property group in each sequence and returns a NumPy array of these proportions and a list of property names.
def AACPCP(fastas, **kw):
    groups = {
        'charged':'DEKHR',
        'aliphatic':'ILV',
        'aromatic':'FHWY',
        'polar':'DERKQN',
        'neutral':'AGHPSTY',
        'hydrophobic':'CVLIMFW',
        'positively-charged':'HKR',
        'negatively-charged':'DE',
        'tiny':'ACDGST',
        'small':'EHILKMNPQV',
        'large':'FRWY'
    }


    property = (
    'charged', 'aliphatic', 'aromatic', 'polar',
    'neutral', 'hydrophobic', 'positively-charged', 'negatively-charged',
    'tiny', 'small', 'large')

    encodings = []
    header = property

    for i in fastas:
        name, sequence = i[0], re.sub('-', '', i[1])
        code = []
        for p in property:
            c = Count(groups[p], sequence) / len(sequence)
            code = code + [c]
        encodings.append(code)
    return np.array(encodings, dtype=float), list(header)

In [13]:
# This function calculates the composition, transition, and distribution of physicochemical properties for the sequences in fastas. It first defines three groups of amino acids for each property. Then, it calculates the proportion of each group for each property in each sequence and returns a NumPy array of these proportions and a list of property names.
def CTDC(fastas, **kw):
    group1 = {
        'hydrophobicity_PRAM900101': 'RKEDQN',
        'hydrophobicity_ARGP820101': 'QSTNGDE',
        'hydrophobicity_ZIMJ680101': 'QNGSWTDERA',
        'hydrophobicity_PONP930101': 'KPDESNQT',
        'hydrophobicity_CASG920101': 'KDEQPSRNTG',
        'hydrophobicity_ENGD860101': 'RDKENQHYP',
        'hydrophobicity_FASG890101': 'KERSQD',
        'normwaalsvolume': 'GASTPDC',
        'polarity':        'LIFWCMVY',
        'polarizability':  'GASDT',
        'charge':          'KR',
        'secondarystruct': 'EALMQKRH',
        'solventaccess':   'ALFCGIVW'
    }
    group2 = {
        'hydrophobicity_PRAM900101': 'GASTPHY',
        'hydrophobicity_ARGP820101': 'RAHCKMV',
        'hydrophobicity_ZIMJ680101': 'HMCKV',
        'hydrophobicity_PONP930101': 'GRHA',
        'hydrophobicity_CASG920101': 'AHYMLV',
        'hydrophobicity_ENGD860101': 'SGTAW',
        'hydrophobicity_FASG890101': 'NTPG',
        'normwaalsvolume': 'NVEQIL',
        'polarity':        'PATGS',
        'polarizability':  'CPNVEQIL',
        'charge':          'ANCQGHILMFPSTWYV',
        'secondarystruct': 'VIYCWFT',
        'solventaccess':   'RKQEND'
    }
    group3 = {
        'hydrophobicity_PRAM900101': 'CLVIMFW',
        'hydrophobicity_ARGP820101': 'LYPFIW',
        'hydrophobicity_ZIMJ680101': 'LPFYI',
        'hydrophobicity_PONP930101': 'YMFWLCVI',
        'hydrophobicity_CASG920101': 'FIWC',
        'hydrophobicity_ENGD860101': 'CVLIMF',
        'hydrophobicity_FASG890101': 'AYHWVMFLIC',
        'normwaalsvolume': 'MHKFRYW',
        'polarity':        'HQRKNED',
        'polarizability':  'KMHFRYW',
        'charge':          'DE',
        'secondarystruct': 'GNPSD',
        'solventaccess':   'MSPTHY'
    }

    groups = [group1, group2, group3]
    property = (
    'hydrophobicity_PRAM900101', 'hydrophobicity_ARGP820101', 'hydrophobicity_ZIMJ680101', 'hydrophobicity_PONP930101',
    'hydrophobicity_CASG920101', 'hydrophobicity_ENGD860101', 'hydrophobicity_FASG890101', 'normwaalsvolume',
    'polarity', 'polarizability', 'charge', 'secondarystruct', 'solventaccess')

    encodings = []
    header = []
    for p in property:
        for g in range(1, len(groups) + 1):
            header.append(p + '.G' + str(g))

    for i in fastas:
        name, sequence = i[0], re.sub('-', '', i[1])
        code = []
        for p in property:
            c1 = Count(group1[p], sequence) / len(sequence)
            c2 = Count(group2[p], sequence) / len(sequence)
            c3 = 1 - c1 - c2
            code = code + [c1, c2, c3]
        encodings.append(code)
    return np.array(encodings, dtype=float), header

In [14]:
# This function takes an amino acid set aaSet and a sequence as input, and returns a list of percentages representing the position of the first occurrence of each cutoff number (1, 25%, 50%, 75%, and 100%) of amino acids from the set within the sequence. If no amino acid from the set is found in the sequence, it appends 0 to the list.
def Count2(aaSet, sequence):
    number = 0
    for aa in sequence:
        if aa in aaSet:
            number = number + 1
    cutoffNums = [1, math.floor(0.25 * number), math.floor(0.50 * number), math.floor(0.75 * number), number]
    cutoffNums = [i if i >=1 else 1 for i in cutoffNums]

    code = []
    for cutoff in cutoffNums:
        myCount = 0
        for i in range(len(sequence)):
            if sequence[i] in aaSet:
                myCount += 1
                if myCount == cutoff:
                    code.append((i + 1) / len(sequence) * 100)
                    break
        if myCount == 0:
            code.append(0)
    return code

In [15]:
# Input: a list of fasta format protein sequences and optional keyword arguments.
# The function computes the Composition, Transition, and Distribution of three groups of amino acids based on different properties like hydrophobicity, normwaalsvolume, polarity, polarizability, charge, secondary structure, and solvent accessibility.
# Returns a NumPy array of calculated encoding values and a header list.
def CTDD(fastas, **kw):
    group1 = {
        'hydrophobicity_PRAM900101': 'RKEDQN',
        'hydrophobicity_ARGP820101': 'QSTNGDE',
        'hydrophobicity_ZIMJ680101': 'QNGSWTDERA',
        'hydrophobicity_PONP930101': 'KPDESNQT',
        'hydrophobicity_CASG920101': 'KDEQPSRNTG',
        'hydrophobicity_ENGD860101': 'RDKENQHYP',
        'hydrophobicity_FASG890101': 'KERSQD',
        'normwaalsvolume': 'GASTPDC',
        'polarity':        'LIFWCMVY',
        'polarizability':  'GASDT',
        'charge':          'KR',
        'secondarystruct': 'EALMQKRH',
        'solventaccess':   'ALFCGIVW'
    }
    group2 = {
        'hydrophobicity_PRAM900101': 'GASTPHY',
        'hydrophobicity_ARGP820101': 'RAHCKMV',
        'hydrophobicity_ZIMJ680101': 'HMCKV',
        'hydrophobicity_PONP930101': 'GRHA',
        'hydrophobicity_CASG920101': 'AHYMLV',
        'hydrophobicity_ENGD860101': 'SGTAW',
        'hydrophobicity_FASG890101': 'NTPG',
        'normwaalsvolume': 'NVEQIL',
        'polarity':        'PATGS',
        'polarizability':  'CPNVEQIL',
        'charge':          'ANCQGHILMFPSTWYV',
        'secondarystruct': 'VIYCWFT',
        'solventaccess':   'RKQEND'
    }
    group3 = {
        'hydrophobicity_PRAM900101': 'CLVIMFW',
        'hydrophobicity_ARGP820101': 'LYPFIW',
        'hydrophobicity_ZIMJ680101': 'LPFYI',
        'hydrophobicity_PONP930101': 'YMFWLCVI',
        'hydrophobicity_CASG920101': 'FIWC',
        'hydrophobicity_ENGD860101': 'CVLIMF',
        'hydrophobicity_FASG890101': 'AYHWVMFLIC',
        'normwaalsvolume': 'MHKFRYW',
        'polarity':        'HQRKNED',
        'polarizability':  'KMHFRYW',
        'charge':          'DE',
        'secondarystruct': 'GNPSD',
        'solventaccess':   'MSPTHY'
    }

    groups = [group1, group2, group3]
    property = (
    'hydrophobicity_PRAM900101', 'hydrophobicity_ARGP820101', 'hydrophobicity_ZIMJ680101', 'hydrophobicity_PONP930101',
    'hydrophobicity_CASG920101', 'hydrophobicity_ENGD860101', 'hydrophobicity_FASG890101', 'normwaalsvolume',
    'polarity', 'polarizability', 'charge', 'secondarystruct', 'solventaccess')


    encodings = []
    header = []
    for p in property:
        for g in ('1', '2', '3'):
            for d in ['0', '25', '50', '75', '100']:
                header.append(p + '.' + g + '.residue' + d)

    for i in fastas:
        name, sequence  = i[0], re.sub('-', '', i[1])
        code = []
        for p in property:
            code = code + Count2(group1[p], sequence) + Count2(group2[p], sequence) + Count2(group3[p], sequence)
        encodings.append(code)
    return np.array(encodings, dtype=float), header

In [16]:
# Input: a list of fasta format protein sequences and optional keyword arguments.
# Similar to the CTDD function, it computes the Composition, Transition, and Distribution of three groups of amino acids. However, it focuses on the transitions between the groups rather than the distribution.
# Returns a NumPy array of calculated encoding values and a header list.
def CTDT(fastas, **kw):
    group1 = {
        'hydrophobicity_PRAM900101': 'RKEDQN',
        'hydrophobicity_ARGP820101': 'QSTNGDE',
        'hydrophobicity_ZIMJ680101': 'QNGSWTDERA',
        'hydrophobicity_PONP930101': 'KPDESNQT',
        'hydrophobicity_CASG920101': 'KDEQPSRNTG',
        'hydrophobicity_ENGD860101': 'RDKENQHYP',
        'hydrophobicity_FASG890101': 'KERSQD',
        'normwaalsvolume': 'GASTPDC',
        'polarity':        'LIFWCMVY',
        'polarizability':  'GASDT',
        'charge':          'KR',
        'secondarystruct': 'EALMQKRH',
        'solventaccess':   'ALFCGIVW'
    }
    group2 = {
        'hydrophobicity_PRAM900101': 'GASTPHY',
        'hydrophobicity_ARGP820101': 'RAHCKMV',
        'hydrophobicity_ZIMJ680101': 'HMCKV',
        'hydrophobicity_PONP930101': 'GRHA',
        'hydrophobicity_CASG920101': 'AHYMLV',
        'hydrophobicity_ENGD860101': 'SGTAW',
        'hydrophobicity_FASG890101': 'NTPG',
        'normwaalsvolume': 'NVEQIL',
        'polarity':        'PATGS',
        'polarizability':  'CPNVEQIL',
        'charge':          'ANCQGHILMFPSTWYV',
        'secondarystruct': 'VIYCWFT',
        'solventaccess':   'RKQEND'
    }
    group3 = {
        'hydrophobicity_PRAM900101': 'CLVIMFW',
        'hydrophobicity_ARGP820101': 'LYPFIW',
        'hydrophobicity_ZIMJ680101': 'LPFYI',
        'hydrophobicity_PONP930101': 'YMFWLCVI',
        'hydrophobicity_CASG920101': 'FIWC',
        'hydrophobicity_ENGD860101': 'CVLIMF',
        'hydrophobicity_FASG890101': 'AYHWVMFLIC',
        'normwaalsvolume': 'MHKFRYW',
        'polarity':        'HQRKNED',
        'polarizability':  'KMHFRYW',
        'charge':          'DE',
        'secondarystruct': 'GNPSD',
        'solventaccess':   'MSPTHY'
    }

    groups = [group1, group2, group3]
    property = (
    'hydrophobicity_PRAM900101', 'hydrophobicity_ARGP820101', 'hydrophobicity_ZIMJ680101', 'hydrophobicity_PONP930101',
    'hydrophobicity_CASG920101', 'hydrophobicity_ENGD860101', 'hydrophobicity_FASG890101', 'normwaalsvolume',
    'polarity', 'polarizability', 'charge', 'secondarystruct', 'solventaccess')

    encodings = []
    header = []
    for p in property:
        for tr in ('Tr1221', 'Tr1331', 'Tr2332'):
            header.append(p + '.' + tr)

    for i in fastas:
        name, sequence = i[0], re.sub('-', '', i[1])
        code = []
        aaPair = [sequence[j:j + 2] for j in range(len(sequence) - 1)]
        for p in property:
            c1221, c1331, c2332 = 0, 0, 0
            for pair in aaPair:
                if (pair[0] in group1[p] and pair[1] in group2[p]) or (pair[0] in group2[p] and pair[1] in group1[p]):
                    c1221 = c1221 + 1
                    continue
                if (pair[0] in group1[p] and pair[1] in group3[p]) or (pair[0] in group3[p] and pair[1] in group1[p]):
                    c1331 = c1331 + 1
                    continue
                if (pair[0] in group2[p] and pair[1] in group3[p]) or (pair[0] in group3[p] and pair[1] in group2[p]):
                    c2332 = c2332 + 1
            code = code + [c1221/len(aaPair), c1331/len(aaPair), c2332/len(aaPair)]
        encodings.append(code)
    return np.array(encodings, dtype=float), header

In [17]:
# Input: a protein sequence, a gap value, a list of features, and a dictionary mapping amino acids to groups.
# The function calculates the K-Spaced Consecutive Triads (KSCT) for the given protein sequence based on the specified gap and features.
# Returns a list of computed values.
def CalculateKSCTriad(sequence, gap, features, AADict):
    res = []
    for g in range(gap + 1):
        myDict = {}
        for f in features:
            myDict[f] = 0

        for i in range(len(sequence)):
            if i + g + 1 < len(sequence) and i + 2 * g + 2 < len(sequence):
                fea = AADict[sequence[i]] + '.' + AADict[sequence[i + g + 1]] + '.' + AADict[
                    sequence[i + 2 * g + 2]]
                myDict[fea] = myDict[fea] + 1

        maxValue, minValue = max(myDict.values()), min(myDict.values())
        for f in features:
            res.append((myDict[f] - minValue) / maxValue)

    return res

In [18]:
# Input: a list of fasta format protein sequences, a gap value, and optional keyword arguments.
# The function computes the K-Spaced Consecutive Triads (KSCT) encoding for each protein sequence in the input list.
# Returns a NumPy array of calculated encoding values and a header list.
def KSCTriad(fastas, gap=0, **kw):
    AAGroup = {
        'g1': 'AGV',
        'g2': 'ILFP',
        'g3': 'YMTS',
        'g4': 'HNQW',
        'g5': 'RK',
        'g6': 'DE',
        'g7': 'C'
    }

    myGroups = sorted(AAGroup.keys())

    AADict = {}
    for g in myGroups:
        for aa in AAGroup[g]:
            AADict[aa] = g

    features = [f1 + '.' + f2 + '.' + f3 for f1 in myGroups for f2 in myGroups for f3 in myGroups]

    encodings = []
    header = ['#']
    for g in range(gap + 1):
        for f in features:
            header.append(f + '.gap' + str(g))


    for i in fastas:
        name, sequence = i[0], re.sub('-', '', i[1])
        code = [name]
        if len(sequence) < 2 * gap + 3:
            print('Error: for "KSCTriad" encoding, the input fasta sequences should be greater than (2*gap+3). \n\n')
            return 0
        code = code + CalculateKSCTriad(sequence, gap, features, AADict)
        encodings.append(code)

    return np.array(encodings, dtype=float), header

In [19]:


# Input: a list of fasta format protein sequences, a gap value, and optional keyword arguments.
# The function computes the Consecutive Triads (CT) encoding for each protein sequence in the input list.
# Returns a NumPy array of calculated encoding values and a header list.
def CTriad(fastas, gap = 0, **kw):
    AAGroup = {
        'g1': 'AGV',
        'g2': 'ILFP',
        'g3': 'YMTS',
        'g4': 'HNQW',
        'g5': 'RK',
        'g6': 'DE',
        'g7': 'C'
    }

    myGroups = sorted(AAGroup.keys())

    AADict = {}
    for g in myGroups:
        for aa in AAGroup[g]:
            AADict[aa] = g

    features = [f1 + '.'+ f2 + '.' + f3 for f1 in myGroups for f2 in myGroups for f3 in myGroups]

    encodings = []
    header = []
    for f in features:
        header.append(f)

    for i in fastas:
        name, sequence = i[0], re.sub('-', '', i[1])
        code = []
        if len(sequence) < 3:
            print('Error: for "CTriad" encoding, the input fasta sequences should be greater than 3. \n\n')
            return 0
        code = code + CalculateKSCTriad(sequence, 0, features, AADict)
        encodings.append(code)

    return np.array(encodings, dtype=float), header

In [20]:
# Function to compute Di-Dipeptide-based Encoding (DDE) for a given set of protein sequences (fastas)
def DDE(fastas, **kw):
    
    # Define the 20 standard amino acids
    AA = 'ACDEFGHIKLMNPQRSTVWY'
    # Define the codon usage for each amino acid
    myCodons = {
        'A': 4,
        'C': 2,
        'D': 2,
        'E': 2,
        'F': 2,
        'G': 4,
        'H': 2,
        'I': 3,
        'K': 2,
        'L': 6,
        'M': 1,
        'N': 2,
        'P': 4,
        'Q': 2,
        'R': 6,
        'S': 6,
        'T': 4,
        'V': 4,
        'W': 1,
        'Y': 2
    }
    # Initialize the encodings and dipeptides
    encodings = []
    diPeptides = ['DDE_'+aa1 + aa2 for aa1 in AA for aa2 in AA]
    header = [] + diPeptides

    # Calculate the transition matrix (myTM) for dipeptides
    myTM = []
    for pair in diPeptides:
        myTM.append((myCodons[pair[0]] / 61) * (myCodons[pair[1]] / 61))
    # Create a dictionary for amino acid indices
    AADict = {}
    for i in range(len(AA)):
        AADict[AA[i]] = i
    # Calculate DDE encoding for each sequence in fastas
    for i in fastas:
        name, sequence = i[0], re.sub('-', '', i[1])
        code = []
        tmpCode = [0] * 400
        for j in range(len(sequence) - 2 + 1):
            tmpCode[AADict[sequence[j]] * 20 + AADict[sequence[j+1]]] = tmpCode[AADict[sequence[j]] * 20 + AADict[sequence[j+1]]] +1
        if sum(tmpCode) != 0:
            tmpCode = [i/sum(tmpCode) for i in tmpCode]

        myTV = []
        for j in range(len(myTM)):
            myTV.append(myTM[j] * (1-myTM[j]) / (len(sequence) - 1))

        for j in range(len(tmpCode)):
            tmpCode[j] = (tmpCode[j] - myTM[j]) / math.sqrt(myTV[j])

        code = code + tmpCode
        encodings.append(code)
    return np.array(encodings, dtype=float), header

# Disorder (Disorder) Protein disorder information was first predicted using external VSL2
# DisorderB also required external program VSL2
# DisorderC also required external program VSL2

In [21]:
# Function to compute Dipeptide Composition (DPC) for a given set of protein sequences (fastas) with a specified gap
def DPC(fastas, gap, **kw):
    AA = 'ACDEFGHIKLMNPQRSTVWY'
    encodings = []
    diPeptides = [aa1 + aa2 for aa1 in AA for aa2 in AA]
    header = [] + diPeptides

    AADict = {}
    for i in range(len(AA)):
        AADict[AA[i]] = i
    
    # Calculate DPC encoding for each sequence in fastas
    for i in fastas:
        name, sequence = i[0], re.sub('-', '', i[1])
        code = []
        tmpCode = [0] * 400
        for j in range(len(sequence) - 2 + 1 - gap):
            tmpCode[AADict[sequence[j]] * 20 + AADict[sequence[j+gap+1]]] = tmpCode[AADict[sequence[j]] * 20 + AADict[sequence[j+gap+1]]] +1
        if sum(tmpCode) != 0:
            tmpCode = [i/sum(tmpCode) for i in tmpCode]
        code = code + tmpCode
        encodings.append(code)
    return np.array(encodings, dtype=float), header

In [22]:
# Function to compute Equal Amino Acid Composition (EAAC) for a given set of protein sequences (fastas) with a specified window size
def EAAC(fastas, window=5, **kw):
    AA ='ACDEFGHIKLMNPQRSTVWY'
    #AA = 'ARNDCQEGHILKMFPSTWYV'
    encodings = []
    header = []
    for aa in AA:
        header.append('EACC.'+aa)
    # Calculate EAAC encoding for each sequence in fastas
    for i in fastas:
        name, sequence = i[0], i[1]
        code = []
        for aa in AA:
            tmp = 0
            for j in range(len(sequence)):
                if j < len(sequence) and j + window <= len(sequence):
                    count = Counter(sequence[j:j+window])
                    for key in count:
                        count[key] = count[key] / len(sequence[j:j+window])
                    tmp = tmp + count[aa]
            code.append(tmp/len(sequence))
        encodings.append(code)
    return np.array(encodings, dtype=float), header

In [23]:
# Function to compute Equal Group Amino Acid Composition (EGAAC) for a given set of protein sequences (fastas) with a specified window size
def EGAAC(fastas, window=5, **kw):
    group = {
        'alphaticr': 'GAVLMI',
        'aromatic': 'FYW',
        'postivecharger': 'KRH',
        'negativecharger': 'DE',
        'uncharger': 'STCPNQ'
    }
    groupKey = group.keys()
    encodings = []
    header = []
    for w in range(1, len(fastas[0][1]) - window + 2):
        for g in groupKey:
            header.append('SW.' + str(w) + '.' + g)

    # Calculate EGAAC encoding for each sequence in fastas
    for i in fastas:
        name, sequence = i[0], i[1]
        code = []
        for key in groupKey:
            tmp=0
            for j in range(len(sequence)):
                if j + window <= len(sequence):
                    count = Counter(sequence[j:j + window])
                    myDict = {}
                    #for key in groupKey:
                    for aa in group[key]:
                        myDict[key] = myDict.get(key, 0) + count[aa]


                    #for key in groupKey:
                    tmp = tmp + (myDict[key] / window)
            code.append(tmp/len(sequence))
        encodings.append(code)
    return np.array(encodings, dtype=float), header

In [24]:
# Function to compute R-value between two amino acids aa1 and aa2 using AADict and Matrix
def Rvalue(aa1, aa2, AADict, Matrix):
    return sum([(Matrix[i][AADict[aa1]] - Matrix[i][AADict[aa2]]) ** 2 for i in range(len(Matrix))]) / len(Matrix)


In [25]:
# Function to compute Pseudo Amino Acid Composition (PAAC) for a given set of protein sequences (fastas) with a specified lambda value and weight (w)
def PAAC(fastas, lambdaValue=5, w=0.05, **kw):
    dataFile = 'data/PAAC.txt'

    with open(dataFile) as f:
        records = f.readlines()
    AA = ''.join(records[0].rstrip().split()[1:])
    AADict = {}
    for i in range(len(AA)):
        AADict[AA[i]] = i
    AAProperty = []
    AAPropertyNames = []
    for i in range(1, len(records)):
        array = records[i].rstrip().split() if records[i].rstrip() != '' else None
        AAProperty.append([float(j) for j in array[1:]])
        AAPropertyNames.append(array[0])

    AAProperty1 = []
    for i in AAProperty:
        meanI = sum(i) / 20
        fenmu = math.sqrt(sum([(j - meanI) ** 2 for j in i]) / 20)
        AAProperty1.append([(j - meanI) / fenmu for j in i])

    encodings = []
    header = []
    for aa in AA:
        header.append('Xc1.' + aa)
    for n in range(1, lambdaValue + 1):
        header.append('Xc2.lambda' + str(n))
        
    # Calculate PAAC encoding for each sequence in fastas
    for i in fastas:
        name, sequence= i[0], re.sub('-', '', i[1])
        code = []
        theta = []
        for n in range(1, lambdaValue + 1):
            theta.append(
                sum([Rvalue(sequence[j], sequence[j + n], AADict, AAProperty1) for j in range(len(sequence) - n)]) / (
                    len(sequence) - n))
        myDict = {}
        for aa in AA:
            myDict[aa] = sequence.count(aa)
        code = code + [myDict[aa] / (1 + w * sum(theta)) for aa in AA]
        code = code + [(w * j) / (1 + w * sum(theta)) for j in theta]
        encodings.append(code)
    return np.array(encodings, dtype=float), header

# Machine Learning
This code is for evaluating the performance of various classifiers on protein sequence data. It consists of two functions: cv and test. The cv function performs cross-validation, and the test function is for testing the performance of classifiers on test data. The main part of the code reads data from protein sequences in FASTA format, extracts features using various feature extraction methods, trains multiple classifiers on these features, and evaluates their performance.

Let's go through the code in detail:

The cv function performs cross-validation on the given classifier (clf), data (X), labels (y), and the number of folds (nr_fold). It first initializes some lists to store performance metrics. Then, it splits the data into training and testing sets based on the fold number, fits the classifier on the training set, and computes various performance metrics (accuracy, sensitivity, specificity, Matthews correlation coefficient, and area under the ROC curve) on the test set. Finally, it returns the mean values of these performance metrics.

The test function trains the given classifier (clf) on the training data (X, y) and evaluates it on the test data (Xt, yt). It calculates the same performance metrics as the cv function and returns them.

The main part of the code starts by reading protein sequences from FASTA files and extracting features using various feature extraction methods, such as AAC, DPC, APAAC, CTDC, CTDD, CTDT, GAAC, KSCTriad, CTriad, DDE, and PAAC. It concatenates the features for both positive and negative samples to create the full feature set and their corresponding labels.

Next, it reads test protein sequences and extracts the same features as for the training data.

The code then trains and evaluates multiple classifiers, including RandomForestClassifier, ExtraTreesClassifier, SVC, LogisticRegression, XGBClassifier, and KNeighborsClassifier, on each of the feature sets (10 in total). It stores the predicted probabilities from these classifiers as additional features in featx.

In summary, this code reads protein sequences, extracts features from them, and evaluates various classifiers on this feature set to predict whether the proteins are part of a specific class or not.





In [26]:
# Function for k-fold cross-validation
def cv(clf, X, y, nr_fold):
    ix = []
    # Create an array with indices of the samples
    for i in range(0, len(y)):
        ix.append(i)
    ix = np.array(ix)

    # Initialize lists to store performance metrics
    allACC = []
    allSENS = []
    allSPEC = []
    allMCC = []
    allAUC = []
    
    # Loop over each fold
    for j in range(0, nr_fold):
        # Create boolean masks for training and testing sets
        train_ix = ((ix % nr_fold) != j)
        test_ix = ((ix % nr_fold) == j)
        
        # Split the dataset into training and testing sets
        train_X, test_X = X[train_ix], X[test_ix]
        train_y, test_y = y[train_ix], y[test_ix]
        
        # Train the classifier
        clf.fit(train_X, train_y)
        
        # Make predictions and probability predictions
        p = clf.predict(test_X)
        pr = clf.predict_proba(test_X)[:,1]
        
        # Initialize variables for confusion matrix elements
        TP=0
        FP=0
        TN=0
        FN=0
        
        # Calculate confusion matrix elements
        for i in range(0,len(test_y)):
            if test_y[i]==0 and p[i]==0:
                TP+= 1
            elif test_y[i]==0 and p[i]==1:
                FN+= 1
            elif test_y[i]==1 and p[i]==0:
                FP+= 1
            elif test_y[i]==1 and p[i]==1:
                TN+= 1
                
        # Calculate performance metrics
        ACC = (TP+TN)/(TP+FP+TN+FN)
        SENS = TP/(TP+FN)
        SPEC = TN/(TN+FP)
        det = math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
        if (det == 0):
            MCC = 0
        else:
            MCC = ((TP*TN)-(FP*FN))/det
        AUC = roc_auc_score(test_y,pr)
        
        # Append performance metrics to the lists
        allACC.append(ACC)
        allSENS.append(SENS)
        allSPEC.append(SPEC)
        allMCC.append(MCC)
        allAUC.append(AUC)
        
    # Return the mean of the performance metrics
    return np.mean(allACC),np.mean(allSENS),np.mean(allSPEC),np.mean(allMCC),np.mean(allAUC)


# Function for evaluating a classifier without cross-validation
def test(clf, X, y, Xt, yt):
    # Assign the training and testing sets
    train_X, test_X = X, Xt
    train_y, test_y = y, yt
    
    # Train the classifier
    clf.fit(train_X, train_y)
    
    # Make predictions and probability predictions
    p = clf.predict(test_X)
    pr = clf.predict_proba(test_X)[:,1]
    
    # Initialize variables for confusion matrix elements
    TP=0
    FP=0
    TN=0
    FN=0
    
    # Calculate confusion matrix elements
    for i in range(0,len(test_y)):
        if test_y[i]==0 and p[i]==0:
            TP+= 1
        elif test_y[i]==0 and p[i]==1:
            FN+= 1
        elif test_y[i]==1 and p[i]==0:
            FP+= 1
        elif test_y[i]==1 and p[i]==1:
            TN+= 1
            
    # Calculate performance metrics
    ACC = (TP+TN)/(TP+FP+TN+FN)
    SENS = TP/(TP+FN)
    SPEC = TN/(TN+FP)
    det = math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
    if (det == 0):
        MCC = 0
    else:
        MCC = ((TP*TN)-(FP*FN))/det
    AUC = roc_auc_score(test_y,pr)

    # Return the performance metrics
    return ACC, SENS, SPEC, MCC, AUC

This code reads protein sequences from FASTA files and calculates various features for each sequence. These features are then concatenated to form a feature vector for each sequence.

The code starts by initializing an empty list called header and reading protein sequences from a file named 'data/TR_P_132.fasta'. Then, it computes different features for these sequences using functions like AAC, DPC, APAAC, CTDC, CTDD, CTDT, GAAC, KSCTriad, CTriad, DDE, and PAAC. The features and their corresponding headers are stored in allfeat_pos and allfeat_head, respectively.

The same process is repeated for another set of protein sequences stored in 'data/TR_N_305.fasta', and the features are stored in allfeat_neg. Then, the positive and negative features are concatenated together, and their corresponding class labels are created. The class labels for positive sequences are set to 0, and for negative sequences, they are set to 1. The final feature matrix X and the class labels y are generated.

This process is then repeated for another set of protein sequences from files named 'data/TS_P_33.fasta' and 'data/TS_N_77.fasta'. The final feature matrix Xt and the class labels yt are generated for these sequences.

In summary, this code reads protein sequences from multiple FASTA files, computes various features for the sequences, and concatenates the features to create feature matrices and corresponding class labels for training and testing purposes.

In [27]:

# AAC, DPC, APAAC,PAAC,DDE,GAAC,KSCtraid, Ctraid, GDPC, CTDC, CTDD, CTDT,
header = []
fasta = read_protein_sequences('data/TR_P_132.fasta')
feat0, h = AAC(fasta)
header.append(h)
allfeat_pos = feat0
allfeat_head = header[0]
feat1, h = DPC(fasta,0)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat1),axis=1)
allfeat_head = allfeat_head + header[1]
feat2, h = APAAC(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat2),axis=1)
allfeat_head = allfeat_head + header[2]
feat3, h = CTDC(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat3),axis=1)
allfeat_head = allfeat_head + header[3]
feat4, h = CTDD(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat4),axis=1)
allfeat_head = allfeat_head + header[4]
feat5, h = CTDT(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat5),axis=1)
allfeat_head = allfeat_head + header[5]
#feat, h = GDPC(fasta)
#header.append(h)
#allfeat_pos = np.concatenate((allfeat_pos,feat),axis=1)
#allfeat_head = allfeat_head + header[6]
feat6, h = GAAC(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat6[:,1:]),axis=1)
allfeat_head = allfeat_head + header[6]
feat7, h = KSCTriad(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat7[:,1:]),axis=1)
allfeat_head = allfeat_head + header[7]
feat8, h = CTriad(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat8),axis=1)
allfeat_head = allfeat_head + header[8]
feat9, h = DDE(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat9),axis=1)
allfeat_head = allfeat_head + header[9]
feat10, h = PAAC(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat10),axis=1)
allfeat_head = allfeat_head + header[10]
CTD_featurP=np.concatenate((feat3,feat4,feat5),axis=1)

fasta = read_protein_sequences('data/TR_N_305.fasta')
feat0, headerx = AAC(fasta)
allfeat_neg = feat0
feat1, headerx = DPC(fasta,0)
allfeat_neg = np.concatenate((allfeat_neg,feat1),axis=1)
feat2, headerx = APAAC(fasta)
allfeat_neg = np.concatenate((allfeat_neg,feat2),axis=1)
feat3, headerx = CTDC(fasta)
allfeat_neg = np.concatenate((allfeat_neg,feat3),axis=1)
feat4, headerx = CTDD(fasta)
allfeat_neg = np.concatenate((allfeat_neg,feat4),axis=1)
feat5, headerx = CTDT(fasta)
allfeat_neg = np.concatenate((allfeat_neg,feat5),axis=1)
#feat, headerx =GDPC(fasta)
#allfeat_neg = np.concatenate((allfeat_neg,feat),axis=1)
feat6, headerx = GAAC(fasta)
allfeat_neg = np.concatenate((allfeat_neg,feat6[:,1:]),axis=1)
feat7, headerx = KSCTriad(fasta)
allfeat_neg = np.concatenate((allfeat_neg,feat7[:,1:]),axis=1)
feat8, headerx = CTriad(fasta)
allfeat_neg = np.concatenate((allfeat_neg,feat8),axis=1)
feat9, headerx = DDE(fasta)
allfeat_neg = np.concatenate((allfeat_neg,feat9),axis=1)
feat10, headerx = PAAC(fasta)
allfeat_neg = np.concatenate((allfeat_neg,feat10),axis=1)
f = []
before = 0
for i in range(0,10):
    after = before + len(header[i])
    f.append(list(range(before, after)))
    before = after
allfeat = np.concatenate((allfeat_pos, allfeat_neg), axis=0)
allclassT = np.concatenate((np.zeros(len(allfeat_pos)), np.ones(len(allfeat_neg))))
X = allfeat
y = allclassT
ix = []
for i in range(0, len(y)):
    ix.append(i)
ix = np.array(ix)
# Generate Test features
header = []
fasta = read_protein_sequences('data/TS_P_33.fasta')
feat0, h = AAC(fasta)
header.append(h)
allfeat_pos = feat0
allfeat_head = header[0]
feat1, h = DPC(fasta,0)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat1),axis=1)
allfeat_head = allfeat_head + header[1]
feat2, h = APAAC(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat2),axis=1)
allfeat_head = allfeat_head + header[2]
feat3, h = CTDC(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat3),axis=1)
allfeat_head = allfeat_head + header[3]
feat4, h = CTDD(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat4),axis=1)
allfeat_head = allfeat_head + header[4]
feat5, h = CTDT(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat5),axis=1)
allfeat_head = allfeat_head + header[5]
#feat, h = GDPC(fasta)
#header.append(h)
#allfeat_pos = np.concatenate((allfeat_pos,feat),axis=1)
#allfeat_head = allfeat_head + header[6]
feat6, h = GAAC(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat6[:,1:]),axis=1)
allfeat_head = allfeat_head + header[6]
feat7, h = KSCTriad(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat7[:,1:]),axis=1)
allfeat_head = allfeat_head + header[7]
feat8, h = CTriad(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat8),axis=1)
allfeat_head = allfeat_head + header[8]
feat9, h = DDE(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat9),axis=1)
allfeat_head = allfeat_head + header[9]
feat10, h = PAAC(fasta)
header.append(h)
allfeat_pos = np.concatenate((allfeat_pos,feat10),axis=1)
allfeat_head = allfeat_head + header[10]


fasta = read_protein_sequences('data/TS_N_77.fasta')
feat0, header = AAC(fasta)
allfeat_neg = feat0
feat1, header = DPC(fasta, 0)
allfeat_neg = np.concatenate((allfeat_neg, feat1), axis=1)
feat2, h = APAAC(fasta)
allfeat_neg = np.concatenate((allfeat_neg, feat2), axis=1)
feat3, h = CTDC(fasta)
allfeat_neg = np.concatenate((allfeat_neg, feat3), axis=1)
feat4, h = CTDD(fasta)
allfeat_neg = np.concatenate((allfeat_neg, feat4), axis=1)
feat5, h = CTDT(fasta)
allfeat_neg = np.concatenate((allfeat_neg, feat5), axis=1)
#feat, h = GDPC(fasta)
#allfeat_neg = np.concatenate((allfeat_neg, feat), axis=1)
feat6, h = GAAC(fasta)
allfeat_neg = np.concatenate((allfeat_neg, feat6[:,1:]), axis=1)
feat7, h = KSCTriad(fasta)
allfeat_neg = np.concatenate((allfeat_neg, feat7[:,1:]), axis=1)
feat8, h = CTriad(fasta)
allfeat_neg = np.concatenate((allfeat_neg, feat8), axis=1)
feat9, h = DDE(fasta)
allfeat_neg = np.concatenate((allfeat_neg, feat9), axis=1)
feat10, h = PAAC(fasta)
allfeat_neg = np.concatenate((allfeat_neg, feat10), axis=1)

allfeat = np.concatenate((allfeat_pos, allfeat_neg), axis=0)
allclass = np.concatenate((np.ones(len(allfeat_pos)), np.zeros(len(allfeat_neg))))

Xt = allfeat
yt = allclass

print(np.shape(allfeat))

(110, 1849)


In [28]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

In [29]:
def get_featx(Xt):
    featx = []

    for i in range(0, 10):
        Xs = X[:, f[i]]
        Xts = Xt[:, f[i]]
        clf = RandomForestClassifier(n_estimators=500, random_state=0)
        clf.fit(Xs, y)
        pr = clf.predict_proba(Xts)[:, 0]
        pr_c = clf.predict(Xts)
        feat = pr
        feat = np.reshape(feat, (len(Xts), 1))
        if (i == 0):
            featx = feat
        else:
              featx = np.concatenate((featx, feat), axis=1)

        clf = ExtraTreesClassifier(n_estimators=500, random_state=0)
        clf.fit(Xs, y)
        pr = clf.predict_proba(Xts)[:, 0]
        pr_c = clf.predict(Xts)
        feat = pr
        feat = np.reshape(feat, (len(Xts), 1))
        featx = np.concatenate((featx, feat), axis=1)

        clf = SVC(probability=True, random_state=0)
        clf.fit(Xs, y)
        ppr = clf.predict_proba(Xts)[:, 0]
        pr_c = clf.predict(Xts)
        feat = pr
        feat = np.reshape(feat, (len(Xts), 1))
        featx = np.concatenate((featx, feat), axis=1)

        clf = LogisticRegression(random_state=0, max_iter=5000)
        clf.fit(Xs, y)
        pr = clf.predict_proba(Xts)[:, 0]
        pr_c = clf.predict(Xts)
        feat = pr
        feat = np.reshape(feat, (len(Xts), 1))
        featx = np.concatenate((featx, feat), axis=1)

        clf = XGBClassifier()
        clf.fit(Xs, y)
        pr = clf.predict_proba(Xts)[:, 0]
        pr_c = clf.predict(Xts)
        feat = pr
        feat = np.reshape(feat, (len(Xts), 1))
        featx = np.concatenate((featx, feat), axis=1)

        clf = KNeighborsClassifier(weights="distance", algorithm="auto")
        clf.fit(Xs, y)
        pr = clf.predict_proba(Xts)[:, 0]
        pr_c = clf.predict(Xts)
        feat= pr
        feat = np.reshape(feat, (len(Xts), 1))
        featx = np.concatenate((featx, feat), axis=1)
    
    return featx

featx = get_featx(Xt)
Test_PF = featx

print(np.shape(Test_PF))

(110, 60)


In [30]:
import pandas as pd
import numpy as np

In [31]:
def apply_mask(featx):
    mask=[2,3,4,5,9,10,12,14,15,18,24,25,28,34,46,47,48,53,56,57]
    Selected_feat = featx[:,mask]
    return Selected_feat

Xt = apply_mask(featx)
yt = allclass

In [32]:
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import roc_curve, auc
# import utils.tools as utils

In [33]:
# Function to convert probability scores to class labels
def categorical_probas_to_classes(p):
    # Use numpy's argmax function to find the index of the highest probability
    # for each row (sample) along the specified axis (axis=1, i.e., columns)
    return np.argmax(p, axis=1)

# Function to convert class labels to one-hot encoded format
def to_categorical(y, nb_classes=None):
    # Convert the input array (y) to a numpy array with datatype 'int'
    y = np.array(y, dtype='int')

    # If the number of classes is not specified, find the highest class label
    # and add 1 to get the total number of classes
    if not nb_classes:
        nb_classes = np.max(y) + 1

    # Create a zero-filled numpy array with the shape (number of samples, number of classes)
    Y = np.zeros((len(y), nb_classes))

    # Iterate through each class label in the input array (y)
    for i in range(len(y)):
        # Set the corresponding element in the one-hot encoded array to 1
        Y[i, y[i]] = 1

    # Return the one-hot encoded array
    return Y

# Function to calculate performance metrics for a binary classification task
def calculate_performace(test_num, pred_y, labels):
    # Initialize true positive (tp), false positive (fp), true negative (tn), and false negative (fn) counts to 0
    tp = 0
    fp = 0
    tn = 0
    fn = 0

    # Iterate through each predicted and true label pair
    for index in range(test_num):
        # If the true label is positive (1)
        if labels[index] == 1:
            # If the prediction is also positive (1), increment the true positive count
            if labels[index] == pred_y[index]:
                tp = tp + 1
            # If the prediction is negative (0), increment the false negative count
            else:
                fn = fn + 1
        # If the true label is negative (0)
        else:
            # If the prediction is also negative (0), increment the true negative count
            if labels[index] == pred_y[index]:
                tn = tn + 1
            # If the prediction is positive (1), increment the false positive count
            else:
                fp = fp + 1

    # Calculate performance metrics using the tp, fp, tn, and fn counts
    acc = float(tp + tn) / test_num  # accuracy
    precision = float(tp) / (tp + fp + 1e-06)  # precision
    npv = float(tn) / (tn + fn + 1e-06)  # negative predictive value
    sensitivity = float(tp) / (tp + fn + 1e-06)  # sensitivity (recall)
    specificity = float(tn) / (tn + fp + 1e-06)  # specificity
    mcc = float(tp * tn - fp * fn) / (math.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)) + 1e-06)  # Matthews correlation coefficient
    # calculates the F1 score, which is a measure of a model's accuracy on a binary classification task. It is the harmonic mean of precision and recall, providing a balance between these two metrics.
    f1 = float(tp * 2) / (tp * 2 + fp + fn + 1e-06)
    
    return acc, precision, npv, sensitivity, specificity, mcc, f1


In [34]:
# Stack the test data vertically
xtest = np.vstack(Xt)
y_test = np.vstack(yt)

# Get the shape of the test data
[m, n] = np.shape(xtest)

# Load the pre-trained model from disk
ldmodel = pickle.load(open("model/pima.pickle_model_svm_PF.dat", "rb"))
print("Loaded model from disk")

# Initialize an empty list to store the separate scores
Sepscores = []

# Calculate the predicted probabilities for each sample in the test data
y_score = ldmodel.predict_proba(xtest)
probablity = y_score[:, 1]
print(probablity)

# Set a threshold for classification
threshold = 0.50

# Print the class labels based on the threshold
for i in range(m):
    if probablity[i] >= threshold:
        print('AMY')
        print('-------------------------------')
    else:
        print('Non_AMY')
        print('-------------------------------')

# Convert the probability scores to class labels
y_class = categorical_probas_to_classes(y_score)

# Calculate the false positive rate (fpr) and true positive rate (tpr) for the ROC curve
fpr, tpr, _ = roc_curve(y_test, y_score[:, 1])

# Calculate the area under the ROC curve (roc_auc)
roc_auc = auc(fpr, tpr)

# Calculate various performance metrics
acc, precision, npv, sensitivity, specificity, mcc, f1 = calculate_performace(len(y_class), y_class, y_test)

# Append the performance metrics to the separate scores list
Sepscores.append([acc, sensitivity, specificity, mcc, roc_auc])

# Print the performance metrics
print('AmyPredFRL_Test_Results:acc=%f,sensitivity=%f,specificity=%f,mcc=%f,roc_auc=%f'
      % (acc, sensitivity, specificity, mcc, roc_auc))

# Calculate the mean of the separate scores
scores = np.array(Sepscores)
result1 = np.mean(scores, axis=0)
H1 = result1.tolist()
Sepscores.append(H1)

# Store the final results in a list
result = Sepscores

# Create a pandas DataFrame to store the results
colum = ['ACC', 'Sn', 'Sp', 'MCC', 'AUC']
data_csv = pd.DataFrame(columns=colum, data=result)

# Save the results to a CSV file
data_csv.to_csv('AmyPredFRL_Test_Results.csv')

Loaded model from disk
[0.96939591 0.96340235 0.8432916  0.57013616 0.79242889 0.93613578
 0.65183562 0.71516303 0.78323884 0.95152008 0.90073541 0.74242483
 0.76343362 0.96275142 0.06687006 0.97843749 0.07528113 0.90504876
 0.90565922 0.96983735 0.89313072 0.96624587 0.66471018 0.95175055
 0.94350782 0.95371278 0.71630446 0.39438815 0.78327564 0.10252572
 0.92534627 0.94705653 0.93306811 0.14076612 0.09428377 0.13179319
 0.08164489 0.09621154 0.06821761 0.05421529 0.19905328 0.09368869
 0.08917437 0.08216754 0.08878637 0.27023395 0.24519135 0.16464773
 0.06010027 0.3579558  0.11129866 0.4547526  0.10849668 0.09087592
 0.0757956  0.07530478 0.10436246 0.68109568 0.18186709 0.10441713
 0.10063966 0.08983897 0.05387055 0.45161475 0.06093643 0.43448213
 0.08812225 0.82899185 0.33678549 0.28292898 0.0888146  0.06280906
 0.09181815 0.08730886 0.12135003 0.13928096 0.07732852 0.08639498
 0.08694462 0.36958821 0.08311997 0.09231518 0.07361113 0.08195284
 0.29091711 0.27788856 0.09179208 0.247

C:\Users\joshf\anaconda3\lib\site-packages\sklearn\base.py:310: UserWarning: Trying to unpickle estimator SVC from version 0.23.1 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  warnings.warn(


# Run AMYPred-FRL
To use the program above to predict whether a given sequence is an amyloid, you need to preprocess the input sequence and feed it to the trained model. Assuming that the input sequence is in the same format as the data used to train the model, you can follow these steps:

Preprocess the input sequence.
Transform the sequence into a suitable format for the model.
Pass the transformed sequence through the model.
Evaluate the output probability and apply the threshold to determine the class label.
## Import from FASTA

In [50]:
from Bio import SeqIO


In [53]:
def preprocess_sequence_from_fasta(fasta_file):
    # Read protein sequences (into original format)
    fasta = read_protein_sequences(fasta_file)
    
    # Save the IDs and description
    metadata = list(SeqIO.parse(fasta_file, "fasta"))
    
    
    # convert fasta ids to integers (might want to change this)
    for i in range(len(fasta)):
        fasta[i][0] = str(i)
        
    # Compute features
    header = []
    features = []

    feat_fns = [AAC, DPC, APAAC, CTDC, CTDD, CTDT, GAAC, KSCTriad, CTriad, DDE, PAAC]
    for feat_fn in feat_fns:
        
        if feat_fn == DPC:
            feat, h = feat_fn(fasta, gap=0)
        else:
            feat, h = feat_fn(fasta)
            # Exclude the first column for GAAC and KSCTriad
            if feat_fn in [GAAC, KSCTriad]:
                feat = feat[:, 1:]
                
            
        header.append(h)
        features.append(feat)
    

    # Combine all features
    allfeat = np.concatenate(features, axis=1)

    return allfeat, fasta, metadata



In [56]:
# Input FASTA file
input_fasta_file = "data/sars-cov-2.fasta"

# Preprocess the input sequence
preprocessed_sequence, fasta, metadata = preprocess_sequence_from_fasta(input_fasta_file)
# print(np.shape(preprocessed_sequence))
# selected feature
selected_features = apply_mask(get_featx(preprocessed_sequence))
# print(np.shape(selected_features))

# Pass the preprocessed_sequence to the model for prediction
y_score = ldmodel.predict_proba(selected_features)
probablity = y_score[:, 1]

# Apply the threshold to determine the class label
threshold = 0.90
for i, prob in enumerate(probablity):
    
    print(metadata[i].description)
    print(metadata[i].seq[0:10] + '...')
    
    predicted_class = "AMY" if prob >= threshold else "Non_AMY"

    # Print the result
    print(f"Predicted class: {predicted_class} ({prob}) \n\n")


sp|P0DTC1|R1A_SARS2 Replicase polyprotein 1a OS=Severe acute respiratory syndrome coronavirus 2 OX=2697049 PE=1 SV=1
MESLVPGFNE...
Predicted class: Non_AMY (0.07854196160776328) 


sp|P0DTC2|SPIKE_SARS2 Spike glycoprotein OS=Severe acute respiratory syndrome coronavirus 2 OX=2697049 GN=S PE=1 SV=1
MFVFLVLLPL...
Predicted class: Non_AMY (0.08998793660107443) 


sp|P0DTC3|AP3A_SARS2 ORF3a protein OS=Severe acute respiratory syndrome coronavirus 2 OX=2697049 GN=3a PE=1 SV=1
MDLFMRIFTI...
Predicted class: Non_AMY (0.31199571900901224) 


sp|P0DTC4|VEMP_SARS2 Envelope small membrane protein OS=Severe acute respiratory syndrome coronavirus 2 OX=2697049 GN=E PE=1 SV=1
MYSFVSEETG...
Predicted class: AMY (0.9440942322412373) 


sp|P0DTC5|VME1_SARS2 Membrane protein OS=Severe acute respiratory syndrome coronavirus 2 OX=2697049 GN=M PE=1 SV=1
MADSNGTITV...
Predicted class: Non_AMY (0.11995682531828883) 


sp|P0DTC6|NS6_SARS2 ORF6 protein OS=Severe acute respiratory syndrome coronavirus 2 OX=26970

## Input protein sequence directly 
in fasta format

In [57]:
def preprocess_sequence_from_string(fasta_string):

    # Read protein sequences
    fasta = read_protein_sequences(input_string=fasta_string)
    
    # convert fasta ids to integers (might want to change this)
    for i in range(len(fasta)):
        fasta[i][0] = str(i)

    # Compute features
    header = []
    features = []

    feat_fns = [AAC, DPC, APAAC, CTDC, CTDD, CTDT, GAAC, KSCTriad, CTriad, DDE, PAAC]
    for feat_fn in feat_fns:
        
        if feat_fn == DPC:
            feat, h = feat_fn(fasta, gap=0)
        else:
            feat, h = feat_fn(fasta)
            # Exclude the first column for GAAC and KSCTriad
            if feat_fn in [GAAC, KSCTriad]:
                feat = feat[:, 1:]
                
            
        header.append(h)
        features.append(feat)

    # Combine all features
    allfeat = np.concatenate(features, axis=1)

    return allfeat, fasta


In [58]:
"""MFVFLVLLPLVSSQCVNLTTRTQLPPAYTNSFTRGVYYPDKVFRSSVLHSTQDLFLPFFS
NVTWFHAIHVSGTNGTKRFDNPVLPFNDGVYFASTEKSNIIRGWIFGTTLDSKTQSLLIV
NNATNVVIKVCEFQFCNDPFLGVYYHKNNKSWMESEFRVYSSANNCTFEYVSQPFLMDLE
GKQGNFKNLREFVFKNIDGYFKIYSKHTPINLVRDLPQGFSALEPLVDLPIGINITRFQT
LLALHRSYLTPGDSSSGWTAGAAAYYVGYLQPRTFLLKYNENGTITDAVDCALDPLSETK
CTLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISN
CVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIAD
YNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPC
NGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNLVKNKCVN
FNFNGLTGTGVLTESNKKFLPFQQFGRDIADTTDAVRDPQTLEILDITPCSFGGVSVITP
GTNTSNQVAVLYQDVNCTEVPVAIHADQLTPTWRVYSTGSNVFQTRAGCLIGAEHVNNSY
ECDIPIGAGICASYQTQTNSPRRARSVASQSIIAYTMSLGAENSVAYSNNSIAIPTNFTI
SVTTEILPVSMTKTSVDCTMYICGDSTECSNLLLQYGSFCTQLNRALTGIAVEQDKNTQE
VFAQVKQIYKTPPIKDFGGFNFSQILPDPSKPSKRSFIEDLLFNKVTLADAGFIKQYGDC
LGDIAARDLICAQKFNGLTVLPPLLTDEMIAQYTSALLAGTITSGWTFGAGAALQIPFAM
QMAYRFNGIGVTQNVLYENQKLIANQFNSAIGKIQDSLSSTASALGKLQDVVNQNAQALN
TLVKQLSSNFGAISSVLNDILSRLDKVEAEVQIDRLITGRLQSLQTYVTQQLIRAAEIRA
SANLAATKMSECVLGQSKRVDFCGKGYHLMSFPQSAPHGVVFLHVTYVPAQEKNFTTAPA
ICHDGKAHFPREGVFVSNGTHWFVTQRNFYEPQIITTDNTFVSGNCDVVIGIVNNTVYDP
LQPELDSFKEELDKYFKNHTSPDVDLGDISGINASVVNIQKEIDRLNEVAKNLNESLIDL
QELGKYEQYIKWPWYIWLGFIAGLIAIVMVTIMLCCMTSCCSCLKGCCSCGSCCKFDEDD
SEPVLKGVKLHYT"""[306:541]

'TLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISN\nCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIAD\nYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPC\nNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNL'

In [59]:
# Input a single protein sequence in FASTA format
print("Please enter your protein sequence:")
fasta_string = input()

def sequence_amyloidogenicity(fasta_string, threshold = 0.90):
    
    #reformat to fasta if necessary
    if fasta_string[0] != '>':
        fasta_string = '>1\n' + fasta_string

    """ Example Input
    DYIINLIIKNL
    """

    # Preprocess the input sequence
    preprocessed_sequence, fasta = preprocess_sequence_from_string(fasta_string)
    # print(np.shape(preprocessed_sequence))
    # selected feature
    selected_features = apply_mask(get_featx(preprocessed_sequence))
    # print(np.shape(selected_features))

    # Pass the preprocessed_sequence to the model for prediction
    y_score = ldmodel.predict_proba(selected_features)
    probablity = y_score[:, 1]

    # Apply the threshold to determine the class label
    for i, prob in enumerate(probablity):

    #     print(fasta[i])

        predicted_class = "AMY" if prob >= threshold else "Non_AMY"

        # Print the result
        print(f"Predicted class: {predicted_class} ({prob}) \n\n")
    
    return prob

sequence_amyloidogenicity(fasta_string)

Please enter your protein sequence:
TLKSFTVEKGIYQTSNFRVQPTESIVRFPNITNLCPFGEVFNATRFASVYAWNRKRISN\nCVADYSVLYNSASFSTFKCYGVSPTKLNDLCFTNVYADSFVIRGDEVRQIAPGQTGKIAD\nYNYKLPDDFTGCVIAWNSNNLDSKVGGNYNYLYRLFRKSNLKPFERDISTEIYQAGSTPC\nNGVEGFNCYFPLQSYGFQPTNGVGYQPYRVVVLSFELLHAPATVCGPKKSTNL
Predicted class: AMY (0.9806428088439684) 




0.9806428088439684

In [39]:
import json

with open('../voc_proteomes.json', 'r') as file:
    voc_proteomes = json.load(file) 
    
for voc in voc_proteomes:
    print(f'{voc} S')
    try:
        seq = voc_proteomes[voc]["S"][0:672]
        print(seq)
        sequence_amyloidogenicity(seq)
    except:
        pass

B.1.1.529.fasta S
MVMFTPLVPFWITIAYIICISTKHFYWFFSNYLKRRVVFNGVSFSTFEEAALCTFLLNKEMYLKLRSDVLLPLTQYNRYLALYNKYKYFSGAMDTTSYREAACCHLAKALNDFSNSGSDVLYQPPQISITSAVLQSGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPGQTFSVLACYNGSPSGVYQCAMRHNFTIKGSFLNGSCGSVGFNIDYDCVSFCYMHHMELPTGVHAGTDLEGNFYGPFVDRQTAQAAGTDTTITVNVLAWLYAAVINGDRWFLNRFTTTLNDFNLVAMKYNYEPLTQDHVDILGPLSAQTGIAVLDMCASLKELLQNGMNGRTILGSALLEDEFTPFDVVRQCSGVTFQSAVKRTIKGTHHWLLLTILTSLLVLVQSTQWSLFFFXYENAFLPFAMGIIAMSAFAMMFVKHKHAFLCLFLLPSLATVAYFNMVYMPASWVMRIMTWLDMVDTSFKLKDCVMYASAVVLLILMTARTVYDDGARRVWTLMNVLTLVYKVYYGNALDQAISMWALIISVTSNYSGVVTTVMFLARGIVFMCVEYCPIFFITGNTLQCIMLVYCFLGYFCTCYFGLFCLLNRY
Predicted class: Non_AMY (0.5146373585504178) 


B.1.1.7.fasta S
MVMFTPLVPFWITIAYIICISTKHFYWFFSNYLKRRVVFNGVSFSTFEEAALCTFLLNKEMYLKLRSDVLLPLTQYNRYLALYNKYKYFSGAMDTTSYREAACCHLAKALNDFSNSGSDVLYQPPQTSITSAVLQSGFRKMAFPSGKVEGCMVQVTCGTTTLNGLWLDDVVYCPRHVICTSEDMLNPNYEDLLIRKSNHNFLVQAGNVQLRVIGHSMQNCVLKLKVDTANPKTPKYKFVRIQPG

Predicted class: Non_AMY (0.5071197084113472) 


